# 1. 建立索引 (Build Index)
單一 Notebook 完成資料庫初始化與階層式索引，不再依賴 `scripts/` 目錄。

流程：
1. 載入環境設定
2. 初始化階層式 Schema
3. 建立索引（單檔或整個資料夾）

需設定環境變數：`PGVECTOR_URL`, `EMBED_API_BASE`, `EMBED_API_KEY`（模型名稱可選 `EMBED_MODEL_NAME`）。

In [ ]:
# Step 1: 載入環境變數與設定
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# 確保專案根目錄在 sys.path（Notebook 跑在 notebooks/ 內，需要手動加入）
repo_root = Path.cwd().resolve().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from rag_system.config import RAGConfig

load_dotenv()
config = RAGConfig.from_env()
config.validate()

print(f"Repo root set: {repo_root}")
print(f"PGVECTOR_URL set: {bool(config.conn_string)}")
print(f"Embed base: {config.embed_api_base}")
print(f"Embed model: {config.embed_model}")
print(f"Verify SSL: {config.verify_ssl}")

## Step 2: 初始化階層式 Schema
直接呼叫 `rag_system.infrastructure.schema`，可重複執行、具備 idempotent。

In [ ]:
from rag_system.infrastructure.schema import init_hierarchical_schema, get_schema_info

schema_ok = init_hierarchical_schema(config.conn_string)
print("Schema initialized:", schema_ok)
print(get_schema_info(config.conn_string))

## Step 3: 準備索引用的 Use Cases
使用 `IndexDocumentUseCase` 與 `BulkIndexUseCase` 直接在 Notebook 執行，不需 scripts/。

In [ ]:
from rag_system.common import LocalApiEmbeddings
from rag_system.application.indexing import EmbeddingService, IndexDocumentUseCase, BulkIndexUseCase
from rag_system.application.chunking import HierarchicalChunker
from rag_system.infrastructure.database import HierarchicalDocumentRepository, VectorStoreRepository

# 初始化核心元件
doc_repo = HierarchicalDocumentRepository(config.conn_string)
vector_repo = VectorStoreRepository(config.conn_string, embedding_dimension=4096)
embed_model = LocalApiEmbeddings(
    api_base=config.embed_api_base,
    api_key=config.embed_api_key,
    model_name=config.embed_model,
    verify_ssl=config.verify_ssl,
)
embedding_service = EmbeddingService(embed_model)
chunker = HierarchicalChunker()

index_use_case = IndexDocumentUseCase(
    doc_repository=doc_repo,
    vector_repository=vector_repo,
    embedding_service=embedding_service,
    chunker=chunker,
)
bulk_index = BulkIndexUseCase(index_use_case)

print("Use cases ready. Change embed_api_base/key/model via config if needed.")

## Step 4: （可選）索引單一檔案
若只想先試單檔，可指定檔案路徑；預設批次索引目錄為 `data/input`，可在 Step 5 調整 `data_root`。

In [ ]:
# 單檔索引（可跳過）
sample_file = Path("data/example.md")
if sample_file.exists():
    index_use_case.execute(sample_file, force_reindex=True)
else:
    print("Sample file not found; skip single-file demo.")

## Step 5: 批次索引整個資料夾（預設步驟）
會自動搜尋常見文字與文件格式，並以 `BulkIndexUseCase` 執行。

In [ ]:
# 預設批次索引 data/input 下所有常見文件格式。
# 以 repo_root 為基準，避免在 notebooks/ 工作目錄找不到檔案。

from rag_system.application.indexing import IndexingLevel


data_root = repo_root / "data/input"
extensions = {".md", ".txt", ".pdf", ".docx", ".rtf"}
files = [p for p in data_root.rglob("*") if p.is_file() and p.suffix.lower() in extensions]
print(f"Found {len(files)} files under {data_root}")

if files:
    # RTF/Office 可能有嵌入物，若遇失敗可調整這裡的 try/except 策略
    bulk_index.execute(files, force_reindex=True, skip_errors=False)
else:
    print("No files found. Add documents under data/input then rerun.")